In [ ]:
!pip install transformers datasets torch
!pip install seqeval  # for evaluating NER performance


In [ ]:
import pandas as pd

# Load the CoNLL format dataset
data = pd.read_csv('path_to_your_conll_dataset.txt', delimiter='\t', header=None)
# Example: ['Token', 'Label'] format


In [ ]:
from transformers import XLMRobertaTokenizer

tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['text'], padding=True, truncation=True, is_split_into_words=True)

    labels = examples['labels']
    tokenized_inputs['labels'] = labels
    return tokenized_inputs

# Assuming you have a 'text' and 'labels' column in your dataset
tokenized_dataset = data.map(tokenize_and_align_labels, batched=True)


Fine-tuning the model

In [ ]:
#Define Training Arguments:

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory for the model
    evaluation_strategy="epoch",     # evaluate after each epoch
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=64,   # batch size for evaluation
    num_train_epochs=3,              # number of training epochs
    weight_decay=0.01,               # strength of weight decay
)


In [ ]:
Initialize the Model:
from transformers import XLMRobertaForTokenClassification

model = XLMRobertaForTokenClassification.from_pretrained('xlm-roberta-base', num_labels=len(label_list))


In [ ]:
Set Up the Trainer:
from transformers import Trainer, TrainingArguments

trainer = Trainer(
    model=model,                         # the model to be fine-tuned
    args=training_args,                  # training arguments
    train_dataset=train_dataset,         # training dataset
    eval_dataset=eval_dataset,           # evaluation dataset
    compute_metrics=compute_metrics,     # evaluation metrics
)


In [ ]:
trainer.train()


Evaluating the model

In [ ]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_labels = labels

    results = []
    for pred, true in zip(predictions, true_labels):
        temp_pred, temp_true = [], []
        for p, t in zip(pred, true):
            if t != -100:  # Skip padding tokens
                temp_pred.append(p)
                temp_true.append(t)
        results.append((temp_pred, temp_true))

    precision = precision_score(results)
    recall = recall_score(results)
    f1 = f1_score(results)
    return {"precision": precision, "recall": recall, "f1": f1}


In [ ]:
trainer.evaluate()


In [ ]:
# Save the Fine-Tuned Model
model.save_pretrained('fine_tuned_model')
tokenizer.save_pretrained('fine_tuned_model')
